# Batch LLM using vLLM and Ray Data

This notebook will provide an example walking through how to use Ray Data's `map_batches` to run batch inference of a LLM.

All dependencies are pre-installed on this cluster. Let's move on to some imports.

In [ ]:
from vllm import LLM, SamplingParams
from typing import Dict
import numpy as np
import ray
import random


Set Up defaults that will be used including
* Which model to load
* The sampling params object used by vLLM.

In [ ]:
# Set to the model that you wish to use. Note that using the llama models will require a hugging face token to be set.
hf_model = "meta-llama/Llama-2-7b-chat-hf"
# Create a sampling params object.
sampling_params = SamplingParams(temperature=0, max_tokens=4096)

Create some sample chat messages for summarization. This can easily be converted to an external datasource in the next code block.

In [ ]:
prefix = """
Summarize the below conversation into bullet points
- do not reply to the message, just summarize the conversation into bullet points
- be concise but make sure to capture all details related customer issues and actions
- output 1-5 bullet points
- each bullet point needs to start with '### '
- prefer to use host/guest/customer following with first name and verb
"""

input_tickets = [
"""
guest(messaging): As I have referenced, I picked the INITIAL location because of its location. I was then given no other option, as a first time Air BnBer, to accept the other location. That location is a substantial inconvience, especially after already having travelled in reliance on my initial scheduled location. At this point I just am asking for a location that is NOT 40 minutes from where I requested.
guest(messaging): Given the sudden shift in location and the behaviors of ER since my request I am completely uncomfortable staying at his properties.
agent(messaging): Hi Arielle,  Good day to you. And I hope you are always doing great. This is Jo, one of the Support Ambassadors in the Resolutions Team from Airbnb.   Please accept my sincerest apologies for the inconvenience you have experienced during your stay. I know how distressing this is for you, but I am here to assure you that I will take personal ownership in resolving this issue for you right away.  If unexpected things like this happen, Airbnb offers AirCover, which includes protection from Host cancellations, listing inaccuracies, and other issues like trouble checking in.  You can learn more about how AirCover protects your booking in our Help Center:  airbnb.com/help/article/3218  In order to take advantage of AirCover protection, guests should: — Report any travel issue to us within 72 hours of discovering it — Have used reasonable efforts to remedy the circumstances with the Host prior to making a claim, We'll verify this in your account. — Share some documentation (photos, videos etc.) of the issue with us within 24hrs.  For more details of AirCover please check our Help Center: airbnb.com/help/article/3218  In relation to this case, may I know the exact location that you were supposed to be in?  Regards,
guest(messaging): <ADDRESS> in celebration area of Orlando but the different location was <ADDRESS> which was double the distance from where we had requested. Resulting in us having to change our entire plan. Our vacation was expected to be in the area of Epcot and Disney Springs.
agent(messaging): We completely understand that, and we do apologize for the inconvenience. Have you contacted the Host about this? Please feel free to raise a resolution center case if you want to receive any kind of compensation. You can learn how to do this in the next link:  https://www.airbnb.com/help/article/767  Let me know if you have any other question.  Regards.
""",

"""
agent(messaging): Hi Dominic,  Thanks for taking the time to reach out. I'm going to forward your case to a member of our team who's in a better position to resolve this for you.   We understand that this may have caused some inconvenience to you Dominic, so we're grateful for your patience.  Best regards,
agent(messaging): Please go through the link change the phone number of the account: https://www.airbnb.com/help/article/2679
agent(messaging): Hi Dominic,  Thank you for the time and patience with us. Initially you have contacted us to reinstate the reservation that is cancelled. As the dates of the reservation has passed you can not book the reservation. Host approved for the Guest to stay in the listing. Guest has cancelled the reservation and did not get any refund as per the cancellation policy. The payout for the Host is yet to be released, as per the Airbnb timeline Host will get the payout.   As you have stayed in the listing and the check out is today, we will leave reservation as it is so that the Host can receive the payout for the reservation. You are worried that you will not be able to give the review to the Host. We are sorry as the reservation is cancelled the review request can not be triggered.  I'm glad that I was able to assist you with your concern and if in any case, you need further assistance, please feel free to reach out to us anytime as we will be glad to help.  regards,  Susan
agent(messaging): Hi Holiways V.,  Thank you for your understanding. We are contacting on behalf  of the guest Dominic R. The reservation <RESERVATION_ID> has been cancelled from the Guest end and the payout will be processed to your account. As the guest can not book the reservation with the dates that are crossed, so we have asked to keep the reservation as it is, you will receive the payout of €663.00 to your payout method. Please go through the link to find the details about the payout timelines: https://www.airbnb.com/help/article/425  Regards,  Susan
agent(messaging): I'm glad I was able to help out. It's been great speaking with you, Holiways—thanks for being a Superhost for our community!   Don't hesitate to let me know if you have any other questions.  Best, Susan S.
""",

"""
agent(messaging): Hi Connie,  I'm Esdras A., your Support Ambassador at Airbnb. I'm reaching out with the unfortunate news that Patrick M the Host of your Williamsburg, VA  reservation (<RESERVATION_ID>) for May 12, check-in had to cancel.    Your refund for the canceled reservation has already been processed, and will usually be deposited in your account within up to 15 business days, depending on how you paid. You can learn more about when you’ll get your refund in our Help Center:  airbnb.com/help/article/1967  With AirCover’s Booking Protection Guarantee, I can support you in finding a similar or better home. The way it works is that I’ll ask for a few more details and any “must-haves.” Then I’ll pull together some options that I think will work great for you. If any of them are more expensive than your original reservation, we’ll try to cover the extra cost for any of these options. At this time, we’ve already sent you a $28.00 coupon which you can use on your next stay with us. The coupon is valid for 30 days.  Coupon code: <COUPON_CODE> Coupon Amount: $28.00 Date of expiry: 2023-06-02         Terms and conditions: This coupon can only be used for reservations that start by 5/2/2024. This coupon can only be applied to reservations of $540 or more, excluding guest fees and taxes.  You can learn more about coupon terms in our Help Center: airbnb.com/help/article/3216  I’m about to give you a call, so we can get started. If you need anything in the meantime, or if you’d prefer messaging to a phone call, just send me a reply here.  If you want to learn more about how AirCover protects your booking, you can check out our Help Center article: airbnb.com/help/article/3218  Looking forward to connecting with you soon!
agent(messaging): Hi Connie. As we discussed, I’ve pulled together 2 great options for a new place to stay that are similar to the one you originally booked.   AirCover’s Booking Protection Guarantee means we’ll find you a similar place to stay, so I've also added a $28 coupon to your account, which may cover the difference in cost for any of these options. The coupon is good for 30 days and will be applied automatically at checkout. You can learn more about coupon terms in our Help Center:   https://airbnb.com/help/article/3216  Your refund is on its way and will usually deposit in your account within 10 days, depending on how you paid.   Go ahead and take your time going through these options. I’m here if you have any questions.
agent(messaging): None
agent(messaging): Please let me know if one of the option may like you so we can work on how we will cover it.
agent(messaging): Sure, then, let me work on the coupon amount so you can book right away, allow me a few minutes please.
guest(messaging): Ok
agent(messaging): The new place total amount will be $732.38, I will do my best so Airbnb cover the new listing.
guest(messaging): Please help this is very disturbing
agent(messaging): Sure, do not worry, I am on it, just be aware that this type of transaction is one time courtesy.
guest(messaging): The point of making reservations early is to get the best deal and I did that going through your company and paid in full
guest(messaging): I did not cancel or did anything wrong
agent(messaging): I definitely understand you Connie, but do not worry, I am here to help you, allow me a few minutes more please.
agent(messaging): I am still working in this transaction Connie, allow me a few minutes more please.
guest(messaging): Ok
agent(messaging): Thank you for waiting Connie, here is the coupon <COUPON_CODE> with the $733.00 amount, please book right away and please let me know if the coupon works. Below the payment option of the listing will be an option that says, coupon code, click on it and use the coupon code. If you have any issues using the coupon code let us know please.
guest(messaging): Ok
agent(messaging): Thank you, Connie, for allowing us to assist you! We hope you have a fantastic stay in Williamsburg, VA. Please don't hesitate to contact us if there's anything else you need or if you have any other questions.
guest(messaging): Thank you
agent(messaging): You Welcome Connie, for now we are going to close the case but please do not hesitate to call us back if you need more help. Best regards.
agent(messaging): Good bye.
""",

"""
agent(messaging): Hi Debra,  I'm Mirna T., your Support Ambassador at Airbnb. I'm reaching out with the unfortunate news that Jancy the Host of your Granbury reservation (<RESERVATION_ID>) for May 26 to 29,  check-in had to cancel.   Your refund for the canceled reservation has already been processed, and will usually be deposited in your account within up to 15 business days, depending on how you paid. You can learn more about when you’ll get your refund in our Help Center:  airbnb.com/help/article/1967  With AirCover’s Booking Protection Guarantee, I can support you in finding a similar or better home. The way it works is that I’ll ask for a few more details and any “must-haves.” Then I’ll pull together some options that I think will work great for you. If any of them are more expensive than your original reservation, we’ll try to cover the extra cost for any of these options. At this time, we’ve already sent you a  $25.00 coupon which you can use on your next stay with us. The coupon is valid for 30 days.  Coupon code: <COUPON_CODE> Coupon Amount:  $25.00 Date of expiry: 2023-06-03 Terms and conditions: This coupon can only be used for reservations that start by 5/3/2024. This coupon can only be applied to reservations of $259 or more, excluding guest fees and taxes. You can learn more about coupon terms in our Help Center: airbnb.com/help/article/3216  I’m about to give you a call, so we can get started. If you need anything in the meantime, or if you’d prefer messaging to a phone call, just send me a reply here.  If you want to learn more about how AirCover protects your booking, you can check out our Help Center article: airbnb.com/help/article/3218  Looking forward to connecting with you soon!
agent(messaging): Hi,  this is MIrna,
agent(messaging): Hi Debra. As we discussed, I’ve found a great option for a new place to stay that’s similar to the one you originally booked.   AirCover’s Booking Protection Guarantee means we’ll find you a similar or better place to stay, so I've also added a $25 coupon to your account, which may cover the difference in cost for this option. The coupon is good for 30 days and will be applied automatically at checkout. You can learn more about coupon terms in our Help Center: https://airbnb.com/help/article/3216  Your refund of $120.99  is on its way and will usually deposit in your account within 15 days,it was sent to your payment method  VISA •••••8023.  Go ahead and take your time reviewing the listing. I’m here if you have any questions.
agent(messaging): None
agent(messaging): 2.  Private room w/ attached bathroom and living room About this place Escape to a peaceful retreat by the lake in our cozy private room. Enjoy comfortable accommodations and access to a clean bathroom. Just minutes away from the water, you can spend your days swimming, fishing, or boating. Explore the surrounding forests and parks on foot or bike. So if you're looking for a comfortable and peaceful retreat close to the lake, look no further than our beautiful private room. We can't wait to welcome you into our home!  https://www.airbnb.com/rooms/843704000421188242
agent(messaging): I will be waiting for your comments.  Mirna,
agent(messaging): I just wanted to check in with you, Debra, to find out if you have any questions regarding the options I sent you or if you’re ready to move forward with any of them.  Let me know if you have any questions, or if you want to review any more rebooking options.
guest(messaging): That's the only one I could find as well last night. However my husband is handicapped and can't go upstairs. It's specifically said that there were stairs.
guest(messaging): What about guesthouse hosted by Ginger?
guest(messaging): I don't see any steps.
agent(messaging): I apologize for the late response. I would encourage you to message the Host and inquire about anything if you aren't sure or if you couldn't find your requirements mentioned in the listing description.
agent(messaging): If you have any questions or need further assistance, please let us know, we're here to help!
agent(messaging): Unfortunately, we couldn't find any available Airbnb option, just the one sent, however, I can also check if we have hotels available with one of our partners. I’ll do my best to find a hotel option that fits your requirements, and I’ll send you any options to review.   Would you like me to check for a hotel option?
guest(messaging): Yes please
agent(messaging): Thanks for letting me know! Give me no more than 15 mins.
agent(messaging): Hi Debra,   I was able to find this option only. (https://www.hoteltonight.com/hotel/plantation-inn-in-granbury-tx). Let me know if it works for you.   However, you can look for a hotel that you like, and we can reimburse you up to a certain amount.   Let me know what works better for you. Looking forward to your reply.   Best,
guest(messaging): What would it cost?
agent(messaging): The reimbursement would be for $78 if you send us the hotel receipt, it can be a screenshot of it.
guest(messaging): How does that work?
agent(messaging): When you book the hotel, you just need to send us the receipt of it. Once you send it, you need to add a payout method on your Airbnb account to receive the money.   Follow these steps on how to add a payout method.   Adding a payout method https://www.airbnb.com/help/article/54  Tap Profile > Account settings > Payments and payouts Tap Payout Methods > Add Payout method Choose your billing country region and choose from the options available to you Tap Continue Add the requested info, then tap Continue  Once your payout method is verified, and the hotel receipt sent, the $78 will be reimbursed. Let me know if you need assistance. Looking forward to your reply.  Best, Elisa A.
""",

"""
guest(messaging): ok
agent(messaging): Hi! My name is Charles N, and I’m with Airbnb community support. I’ll be helping you today. Give me a moment while I look into your case.
agent(messaging): Hi, hope you are doing welll. Thanks for reaching out to us. Can you please let me know more about tha nature of your concern
guest(messaging): I am booking an apartment for an employee. When it came to the part for ID and verification. I entered his information, since he is going to be staying there. It told me the request was cancelled. I sent him the link to upload and enter his DL and Photo. He did as we were on the phone. It never gave me a confirmation. I came into work this morning and it said the reservation was cancelled.
agent(messaging): I see. By any chance, do you know what was the confirmation code for the booking that got cancelled?
guest(messaging):<RESERVATION_ID>
agent(messaging): Thanks for letting me know. As per checking here on my end, it seems like the reservation did not get cancelled as it was the request got withdrawn and it did not get the chance to get accepted by the Host.  Furthermore, you may want to consider to try again and book this property as you can do that due to the first request got withdrawn.
guest(messaging): The request was never withdrawn. He uploaded everything  for the verification and received a completed status. not sure where the mix up happened. I will try again.
agent(messaging): I do apologize but based on our system, the request got withdrawn, and the verification is still not attempted.
agent(messaging): I just want to know if we are still connected? This interaction is live, and we haven't heard back from you in the last few minutes. I’ll be available for the next 5 minutes to make sure we’ve discussed your concerns. If we don’t hear from you, we’ll close this chat.
guest(messaging): Brian is in another office in Heflin, AL. How can I send him a link to take a picture with the webcam or mobile app? I am in Louisiana.
agent(messaging): I see. In this case, you may want to consider informing Brian to just create his own Airbnb account as you guys might encounter some trouble specially the verification will be done under Brian's information and you do have access to his account.   Once Brian created his own Airbnb account, inform him that he needs to add you as an additional guest to the reservation so the reservation will also show in your account, that you need to create as well.
agent(messaging): I don't intend to rush you. However, I am from live chat team, and I am not able to stay longer on this thread. I’ll be available for the next 5 minutes to see if we are still connected. If we don’t hear from you, Airbnb will close this chat.
guest(messaging): I am trying to figure this out.
guest(messaging): I understand the verification process but now it will not let me change the phone number to a cell phone instead of my work line.
guest(messaging): I am working it on my cellphone.
agent(messaging): I see. If you are planning to fully verify this account, I suggest make it your ID's and information not Brian as you guys might encounter some trouble in the future if Brian verified this account using his ID's and information, and you do have access to the account.
agent(messaging): No problem, please do take your time.
agent(messaging): I don't intend to rush you. However, I am from live chat team, and I am not able to stay longer on this thread. I’ll be available for the next 5 minutes to see if we are still connected. If we don’t hear from you, Airbnb will close this chat.
guest(messaging): I just sent the pictures.
guest(messaging): I the ID portion is saying that they are unable to confirm??
agent(messaging): As per checking here on my end, it seems like you just have submitted your government ID, which is just a part of the verification process. To complete the verification, you may click this link below as it will route you and guide you to finish it.  www.airbnb.com/verify
guest(messaging): I uploaded the picture to the mobile app.
guest(messaging): It is not sending the text to my cellphone like it says it did.
agent(messaging): Do you mean the code when completing the verification process?
guest(messaging): I am on the mobile app as well.
guest(messaging): yes
guest(messaging): don't have a webcam
agent(messaging): May I know what is the phone number you are using?
guest(messaging): <PHONE_NUMBER>
agent(messaging): I see. Are you trying to verify your phone number or your identity?
guest(messaging): I already verified my phone number. I was able to change it on the mobile app. I just sent another picture. They said lighting was not good.
agent(messaging): I see. So this is not about the sms code. In this case, you may want to re-try it again and make sure that it is clear and bright enough so that our system can read it and evaluate to get your identity verified.
guest(messaging): I did that.
guest(messaging): When I click on the Air BNB that I am trying to book. It is still telling to do the verification. I haven't received another email yet.
agent(messaging): Okay I understand. However, you did not manage to complete yet the verification process. You may want to consider trying it again as there is no error yet as it seems like it's just the lightning of the photo you are submitting.
agent(messaging): You may navigate to your profile and click your personal information, so you can continue verifying your identity as submitting a government ID is just part of the process.
guest(messaging): None
guest(messaging): How can I need more lighting?
guest(messaging): None
agent(messaging): That is not how you will submit or finish the verification process. As per checking, you just putted it in your profile picture, not in the verification process.
guest(messaging): I did add my picture to the profile but I also did the verification process. They are telling me the picture is difficult to see??
agent(messaging): I do apologize as I need to send this case that we have to our specialized team, so they can delete this case and create a new one where they will continue to assist you due to you sent us your driver's license. We do not recommend sending any of your sensitive information through the chat thread as we do not keep it and do not want your personal information get public.
guest(messaging): I am not understanding why they are telling me that the pictures are different.
agent(messaging): No worries as our specialized team will be assisting you further right after creating a new case as they will delete this case first where we are having our conversation as it consist of your driver's license which we do not keep.
agent(messaging): I am forwarding your case to the dedicated team that will be able to assist you. While response times may vary, they will respond in a timely manner.   You may now close the Airbnb app, but please make sure notifications are enabled so that you're alerted when they'll reply.  Here’s a link to Help Center with detailed instruction on how to set and manage notification:  airbnb.com/help/article/14
guest(messaging): I gave your people my employees direct phone number so we can get this done. He is now getting the run around as well.
agent(messaging): Hi, Brian!  I noticed that your account is already verified. Do you need further assistance?
guest(messaging): No. Thank you.
agent(messaging): Thank you so much also! I'm glad I was able to help you out today.   If any concern arises, do not hesitate to get in touch with us.  Best, Mikaela
""",
]

# Set up the prompts to be passed in to the batch inference.
prompts = [prefix + ticket for ticket in input_tickets]

prompts = prompts * 100
random.shuffle(prompts)

Use Ray Data to create a dataset. This notebook will use local text data defined above, but it can easily be expanded to an external source stored in an cloud bucket. To read from an external datasource, you can modify the dataset to be:

```python
input_path = "s3://anonymous@air-example-data/prompts.txt"
ds = ray.data.read_text(
    input_path,
    parallelism=10,
)
```

In [ ]:
ds = ray.data.from_items(
    prompts,
    parallelism=10
)

# alternative, reading from an S3 bucket.
# input_path = "s3://anonymous@air-example-data/prompts.txt"
# ds = ray.data.read_text(
#     input_path,
#     parallelism=10,
# )

Create a utility function fo the batch prediction.

In [ ]:
# Create a predictor class using
class LLMPredictor:
    def __init__(self):
        # Create an LLM.
        self.llm = LLM(model=hf_model)

    def __call__(self, batch: Dict[str, np.ndarray]) -> Dict[str, list]:
        # Generate texts from the prompts. The output is a list of RequestOutput objects
        # that contain the prompt, generated text, and other information.

        outputs = self.llm.generate(batch["item"], sampling_params)
        prompt = []
        generated_text = []
        for i, output in enumerate(outputs):
            prompt.append(output.prompt)
            generated_text.append(' '.join([o.text for o in output.outputs]))
        return {
            "prompt": prompt,
            "generated_text": generated_text,
        }

Run the inference using `map_batches`!

Set the `concurrency` to the number of GPUs available in this cluster if the model fits onto 1 GPU. In this case, the Compute Config uses an advanced Smart Instance Manager configuration to launch 10 GPUs.

In [ ]:
# Apply batch inference for all input data.
ds = ds.map_batches(
    LLMPredictor,
    # Set the concurrency to the number of Ray workers.
    concurrency=10,
    # Specify the number of GPUs required per LLM instance.
    num_gpus=1,
    # Specify the batch size for inference.
    batch_size=32,
)

Time to execute and view the results.

In [ ]:
# ds.show()
ds.take_all()

## Convert to an Anyscale Job

To convert this to an Anyscale Job, you can use the `batch_llm.py` as a starting point. It contains the same code, but uses data from an S3 bucket for the prompts.